#Introduction

Tire selection plays a key role in ensuring road safety, especially in extreme weather conditions. The type of tires used on your vehicle depends on the season and weather conditions. Summer tires offer excellent traction in hot, dry conditions, while winter tires offer better traction in cold, ice, and snow. All-season tires are designed to perform well in both summer and winter. 
 
The aim of this project is to develop a model for identifying the tire type, which distinguishes between summer, winter and all-season tires in particular. The data set used to train and test the model will contain images of tires with different tire wear patterns. 
 
 The development of an accurate tire type recognition model could make a significant contribution to road safety by providing drivers with an automated means of ensuring they are using the correct tires for  current weather conditions. This project aims to harness the power of deep learning to create an efficient and effective tire type recognition system.

## Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Opony_Computer_Vision

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Opony_Computer_Vision


In [2]:
import requests
from bs4 import BeautifulSoup
import time
import  random
import os
from io import BytesIO
from PIL import Image
import cv2
import numpy as np

from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers
from tensorflow.keras.utils import array_to_img, img_to_array, load_img

## Scraping test

In [ ]:
summer = '1'
winter = '2'
both = '3'
page = '1'


url = f'https://www.24opony.pl/szukaj.html?f=1&idproducent=0&idsezon={summer}&idrozmiar=0&idpojazd=1&czyxl=0&czyrf=0&inpre=0&cod=0&cdo=0&strona={page}&idmodel=0'

content = requests.get(url).content
soup = BeautifulSoup(content, 'html.parser')

In [ ]:
images = ['https://www.24opony.pl' + image.find('img')['src'] for image in soup.find_all('a', {'rel':'prettyPhoto'})]

In [ ]:
images[0].split('/')[-1].split('.')[0]

'n_4164_s'

In [ ]:
images

In [ ]:
content = requests.get(images[0]).content

In [ ]:
img  = Image.open(BytesIO(content))

In [ ]:
img.max()

## Scraping

In [ ]:
def get_page(season, page_number):
  url = f'https://www.24opony.pl/szukaj.html?f=1&idproducent=0&idsezon={season}&idrozmiar=0&idpojazd=1&czyxl=0&czyrf=0&inpre=0&cod=0&cdo=0&strona={page_number}&idmodel=0'
  try:
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')
    time.sleep(0.5)
    return content
  except Exception as e:
    print(f'Error: {e}')

def get_urls(content):
  soup = BeautifulSoup(content, 'html.parser')
  return set(['https://www.24opony.pl' + image.find('img')['src'] for image in soup.find_all('a', {'rel':'prettyPhoto'}) if not image.find('img')['src'].split('/')[-1].split('.')[0].startswith('brak_zdjecia')])


def sharpen_image(func):
  def wrapper(url):
    img = func(url)
    new_img = cv2.filter2D(img, -1, kernel=np.array([[0, -1, 0],
                                                [-1, 5, -1],
                                                [0, -1, 0]]))
    return new_img
  return wrapper
    
@sharpen_image
def get_image(url):
  try:
    content = requests.get(url).content
    img = Image.open(BytesIO(content))
    img = np.array(img)
    return img
  except Exception as e:
    print(f'Error: {e}')

def save_image(img, url,season, path):
  file_name = url.split('/')[-1].split('.')[0] + '.jpg'
  os.makedirs(f'{path}/{season}', exist_ok=True)
  cv2.imwrite(f'{path}/{season}/{file_name}', img)

def get_data(season, start_page, end_page, path):
  for page_number in range(start_page, end_page):
    content = get_page(season, page_number + 1)
    urls = get_urls(content)

    for i, url in enumerate(urls):
      img = get_image(url)      
      save_image(img, url, season, path)
      time.sleep(1.0/100)
      if  i % 5 == 0:
        print(f'Download {i} images from page {page_number}/{end_page}')



In [ ]:
#dodaj lsitę z linkami, tak że gdy się link powtarza to skipuje

get_data(1, 1, 400, 'data/train')
get_data(2, 1, 400, 'data/train')
get_data(3, 1, 400, 'data/train')

In [ ]:
def get_page(season, page_number):
  url = f'https://www.24opony.pl/szukaj.html?f=1&idproducent=0&idsezon={season}&idrozmiar=0&idpojazd=1&czyxl=0&czyrf=0&inpre=0&cod=0&cdo=0&strona={page_number}&idmodel=0'
  try:
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')
    time.sleep(0.5)
    return content
  except Exception as e:
    print(f'Error: {e}')

def get_urls(content):
  soup = BeautifulSoup(content, 'html.parser')
  return set(['https://www.24opony.pl' + image['href'] for image in soup.find_all('a', {'rel':'prettyPhoto'}) if not image['href'].split('/')[-1].split('.')[0].startswith('brak_zdjecia')])


def sharpen_image(func):
  def wrapper(url):
    img = func(url)
    new_img = cv2.filter2D(img, -1, kernel=np.array([[0, -1, 0],
                                                [-1, 5, -1],
                                                [0, -1, 0]]))
    return new_img
  return wrapper
    
@sharpen_image
def get_image(url):
  try:
    content = requests.get(url).content
    img = Image.open(BytesIO(content))
    img = np.array(img)
    return img
  except Exception as e:
    print(f'Error: {e}')

def save_image(img, url,season, path):
  file_name = url.split('/')[-1].split('.')[0] + '.jpg'
  os.makedirs(f'{path}/{season}', exist_ok=True)
  cv2.imwrite(f'{path}/{season}/{file_name}', img)

def get_data(season, start_page, end_page, path):
  for page_number in range(start_page, end_page):
    content = get_page(season, page_number + 1)
    urls = get_urls(content)

    for i, url in enumerate(urls):
      img = get_image(url)      
      save_image(img, url, season, path)
      time.sleep(1.0/100)
      if  i % 5 == 0:
        print(f'Download {i} images from page {page_number}/{end_page}')


In [ ]:
get_data(1, 1, 400, 'data/train_various_sizes')
get_data(2, 1, 400, 'data/train_various_sizes')
get_data(3, 1, 400, 'data/train_various_sizes')

#Model

## First model

The model is based on thumbnail photos in which the tread is barely visible. The images have been sharpened.

In [ ]:
test_size = 0.2
path = 'data'

# for sub_type in ['1', '2', '3']:
#   os.makedirs(path + '/test' +f'/{sub_type}', exist_ok=True)
#   file_list = os.listdir(path + '/train/' + sub_type)
#   number_of_samples = int(len(file_list) * test_size)

#   for file_name in random.sample(file_list, number_of_samples):
#     os.rename(path +'/train/' + sub_type + '/' + file_name,
#               path +'/test/' + sub_type + '/' + file_name)

# train_path = 'data/train'
# test_path = 'data/test'
# print(len(os.listdir(train_path + '/1')))
# print(len(os.listdir(test_path + '/1')))

In [ ]:
data_path = 'data/train'

batch_size = 128
img_heights = 128
img_width = 128



In [ ]:
train_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='training',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                    interpolation='bilinear',
                                                    seed=42)

val_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                  interpolation='bilinear',
                                                  seed=42)

Found 1146 files belonging to 3 classes.
Using 917 files for training.
Found 1146 files belonging to 3 classes.
Using 229 files for validation.


In [ ]:
for img, label in train_ds:
  print(img.shape)
  print(label.shape)

(128, 128, 128, 3)
(128, 3)
(128, 128, 128, 3)
(128, 3)
(128, 128, 128, 3)
(128, 3)
(128, 128, 128, 3)
(128, 3)
(128, 128, 128, 3)
(128, 3)
(128, 128, 128, 3)
(128, 3)
(128, 128, 128, 3)
(128, 3)
(21, 128, 128, 3)
(21, 3)


In [ ]:
model = Sequential()

model.add(layers.Rescaling(1/255)) # przez to, z wczytujemy batchami trzeba ustandaryzować używajać warstwy Rescaling
model.add(layers.Conv2D(32,
                        (3, 3),
                        padding='same',
                        activation='relu',
                        input_shape=(img_width, img_heights, 3)))

model.add(layers.MaxPool2D(2,2))

model.add(layers.Conv2D(16,
                        (3, 3),
                        padding='same',
                        activation='relu'
                        ))

model.add(layers.MaxPool2D(2,2))

model.add(layers.Flatten())

model.add(layers.Dense(128,
                       activation='relu'))

model.add(layers.Dense(3,
                       activation='softmax'))

In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    batch_size=128)

Epoch 1/10
8/8 [==============================] - 295s 8s/step - loss: 2.6057 - accuracy: 0.4471 - val_loss: 1.2066 - val_accuracy: 0.3188
Epoch 2/10
8/8 [==============================] - 2s 95ms/step - loss: 1.1318 - accuracy: 0.3740 - val_loss: 1.0805 - val_accuracy: 0.4716
Epoch 3/10
8/8 [==============================] - 2s 82ms/step - loss: 1.0602 - accuracy: 0.5398 - val_loss: 1.0608 - val_accuracy: 0.4716
Epoch 4/10
8/8 [==============================] - 2s 84ms/step - loss: 1.0052 - accuracy: 0.5398 - val_loss: 1.1031 - val_accuracy: 0.4716
Epoch 5/10
8/8 [==============================] - 2s 82ms/step - loss: 0.9893 - accuracy: 0.5398 - val_loss: 1.0454 - val_accuracy: 0.4716
Epoch 6/10
8/8 [==============================] - 2s 81ms/step - loss: 0.9873 - accuracy: 0.5398 - val_loss: 1.0453 - val_accuracy: 0.4716
Epoch 7/10
8/8 [==============================] - 2s 117ms/step - loss: 0.9774 - accuracy: 0.5398 - val_loss: 1.0670 - val_accuracy: 0.4716
Epoch 8/10
8/8 [==========

## Second model

A model to evaluate the impact of image quality on model efficiency. Images resolution and dimensions are higher.

### First set

The same conditions as for the previous model. An improvement in the model's efficiency can be seen, which means that the quality of the photo, and in particular the tread outlines clearly, affect the detection of the right type of tires.

In [ ]:
# test_size = 0.2
# path = 'data'

# for sub_type in ['1', '2', '3']:
#   os.makedirs(path + '/test_various_sizes' +f'/{sub_type}', exist_ok=True)
#   file_list = os.listdir(path + '/train_various_sizes/' + sub_type)
#   number_of_samples = int(len(file_list) * test_size)

#   for file_name in random.sample(file_list, number_of_samples):
#     os.rename(path +'/train_various_sizes/' + sub_type + '/' + file_name,
#               path +'/test_various_sizes/' + sub_type + '/' + file_name)

# train_path = 'data/train_various_sizes'
# test_path = 'data/test_various_sizes'
# print(len(os.listdir(train_path + '/1')))
# print(len(os.listdir(test_path + '/1')))

603
150


In [ ]:
data_path = 'data/train_various_sizes'

batch_size = 128
img_heights = 128
img_width = 128



In [ ]:
train_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='training',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                    interpolation='bilinear',
                                                    seed=42)

val_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                  interpolation='bilinear',
                                                  seed=42)

Found 1145 files belonging to 3 classes.
Using 916 files for training.
Found 1145 files belonging to 3 classes.
Using 229 files for validation.


In [ ]:
model_1 = Sequential()

model_1.add(layers.Rescaling(1/255)) # przez to, z wczytujemy batchami trzeba ustandaryzować używajać warstwy Rescaling
model_1.add(layers.Conv2D(32,
                        (3, 3),
                        padding='same',
                        activation='relu',
                        input_shape=(img_width, img_heights, 3)))

model_1.add(layers.MaxPool2D(2,2))

model_1.add(layers.Conv2D(16,
                        (3, 3),
                        padding='same',
                        activation='relu'
                        ))

model_1.add(layers.MaxPool2D(2,2))

model_1.add(layers.Flatten())

model_1.add(layers.Dense(128,
                       activation='relu'))

model_1.add(layers.Dense(3,
                       activation='softmax'))

In [ ]:
model_1.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history_1 = model_1.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    batch_size=128)

Epoch 1/10
8/8 [==============================] - 5s 151ms/step - loss: 2.2052 - accuracy: 0.4072 - val_loss: 1.1073 - val_accuracy: 0.4716
Epoch 2/10
8/8 [==============================] - 4s 131ms/step - loss: 1.0406 - accuracy: 0.5404 - val_loss: 1.0569 - val_accuracy: 0.4716
Epoch 3/10
8/8 [==============================] - 3s 132ms/step - loss: 0.9998 - accuracy: 0.5404 - val_loss: 1.0463 - val_accuracy: 0.4716
Epoch 4/10
8/8 [==============================] - 3s 139ms/step - loss: 0.9857 - accuracy: 0.5404 - val_loss: 1.0627 - val_accuracy: 0.4716
Epoch 5/10
8/8 [==============================] - 4s 188ms/step - loss: 0.9645 - accuracy: 0.5404 - val_loss: 1.0528 - val_accuracy: 0.4716
Epoch 6/10
8/8 [==============================] - 4s 129ms/step - loss: 0.9468 - accuracy: 0.5622 - val_loss: 1.0137 - val_accuracy: 0.4716
Epoch 7/10
8/8 [==============================] - 3s 135ms/step - loss: 0.9158 - accuracy: 0.5404 - val_loss: 0.9871 - val_accuracy: 0.4760
Epoch 8/10
8/8 [====

### Second set

The number of epochs was selected as the first hyperparameter. However, the result on the test set indicates an accuracy of the magnitude of 100%, while on the validation set it is about 65%. This means that the model tends to overfitting very strongly. So the number of observations should be increased.

In [ ]:
data_path = 'data/train_various_sizes'

batch_size = 128
img_heights = 128
img_width = 128



In [ ]:
train_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='training',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                    interpolation='bilinear',
                                                    seed=42)

val_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                  interpolation='bilinear',
                                                  seed=42)

Found 1145 files belonging to 3 classes.
Using 916 files for training.
Found 1145 files belonging to 3 classes.
Using 229 files for validation.


In [ ]:
model_2 = Sequential()

model_2.add(layers.Rescaling(1/255)) # przez to, z wczytujemy batchami trzeba ustandaryzować używajać warstwy Rescaling
model_2.add(layers.Conv2D(32,
                        (3, 3),
                        padding='same',
                        activation='relu',
                        input_shape=(img_width, img_heights, 3)))

model_2.add(layers.MaxPool2D(2,2))

model_2.add(layers.Conv2D(16,
                        (3, 3),
                        padding='same',
                        activation='relu'
                        ))

model_2.add(layers.MaxPool2D(2,2))

model_2.add(layers.Flatten())

model_2.add(layers.Dense(128,
                       activation='relu'))

model_2.add(layers.Dense(3,
                       activation='softmax'))

In [ ]:
model_2.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history_2 = model_2.fit(train_ds,
                    validation_data=val_ds,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
8/8 [==============================] - 5s 153ms/step - loss: 2.4382 - accuracy: 0.4825 - val_loss: 1.2260 - val_accuracy: 0.4716
Epoch 2/20
8/8 [==============================] - 4s 232ms/step - loss: 1.0845 - accuracy: 0.5404 - val_loss: 1.0740 - val_accuracy: 0.4716
Epoch 3/20
8/8 [==============================] - 3s 135ms/step - loss: 1.0295 - accuracy: 0.5404 - val_loss: 1.0941 - val_accuracy: 0.4716
Epoch 4/20
8/8 [==============================] - 3s 133ms/step - loss: 0.9867 - accuracy: 0.5404 - val_loss: 1.0504 - val_accuracy: 0.4716
Epoch 5/20
8/8 [==============================] - 3s 132ms/step - loss: 0.9679 - accuracy: 0.5404 - val_loss: 1.0968 - val_accuracy: 0.4716
Epoch 6/20
8/8 [==============================] - 5s 140ms/step - loss: 0.9553 - accuracy: 0.5426 - val_loss: 1.0395 - val_accuracy: 0.4803
Epoch 7/20
8/8 [==============================] - 3s 133ms/step - loss: 0.9324 - accuracy: 0.5557 - val_loss: 1.0193 - val_accuracy: 0.5022
Epoch 8/20
8/8 [====

### Images augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,  
    horizontal_flip=True, 
    vertical_flip=True  
)

for sub_type in ['1', '2', '3']:
  # load images from a directory
  image_folder = f'data/raw/{sub_type}'
  save_folder = f'data/train_augmented/{sub_type}'
  if not os.path.exists(save_folder):
    os.makedirs(save_folder)

  for filename in os.listdir(image_folder):
      img = load_img(os.path.join(image_folder, filename))  
      x = img_to_array(img)  
      x = x.reshape((1,) + x.shape)  
      i = 0
      for batch in datagen.flow(x, batch_size=1, save_to_dir=save_folder, save_prefix='aug', save_format='jpg'):
          i += 1
          if i > 9:  
              break  

In [ ]:
w

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=45,  
    horizontal_flip=True, 
    vertical_flip=True  
)

for sub_type in ['1', '2', '3']:
  # load images from a directory
  image_folder = f'data/raw/{sub_type}'
  save_folder = f'data/train_augmented_2/{sub_type}'
  if not os.path.exists(save_folder):
    os.makedirs(save_folder)

  for filename in os.listdir(image_folder):
      img = load_img(os.path.join(image_folder, filename))  
      x = img_to_array(img)  
      x = x.reshape((1,) + x.shape)  
      i = 0
      for batch in datagen.flow(x, batch_size=1, save_to_dir=save_folder, save_prefix='aug', save_format='jpg'):
          i += 1
          if i > 25:  
              break  

#### First set

In [ ]:
# test_size = 0.2
# path = 'data'

# for sub_type in ['1', '2', '3']:
#   os.makedirs(path + '/test_augmented' +f'/{sub_type}', exist_ok=True)
#   file_list = os.listdir(path + '/train_augmented/' + sub_type)
#   number_of_samples = int(len(file_list) * test_size)

#   for file_name in random.sample(file_list, number_of_samples):
#     os.rename(path +'/train_augmented/' + sub_type + '/' + file_name,
#               path +'/test_augmented/' + sub_type + '/' + file_name)

# train_path = 'data/train_augmented'
# test_path = 'data/test_augmented'
# print(len(os.listdir(train_path + '/1')))
# print(len(os.listdir(test_path + '/1')))

4227
1038


In [ ]:
data_path = 'data/train_augmented'

batch_size = 128
img_heights = 128
img_width = 128



In [ ]:
train_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='training',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                    interpolation='bilinear',
                                                    seed=42)

val_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                  interpolation='bilinear',
                                                  seed=42)

Found 8919 files belonging to 3 classes.
Using 7136 files for training.
Found 8919 files belonging to 3 classes.
Using 1783 files for validation.


In [ ]:
model_3 = Sequential()

model_3.add(layers.Rescaling(1/255)) # przez to, z wczytujemy batchami trzeba ustandaryzować używajać warstwy Rescaling
model_3.add(layers.Conv2D(32,
                        (3, 3),
                        padding='same',
                        activation='relu',
                        input_shape=(img_width, img_heights, 3)))

model_3.add(layers.MaxPool2D(2,2))

model_3.add(layers.Conv2D(16,
                        (3, 3),
                        padding='same',
                        activation='relu'
                        ))

model_3.add(layers.MaxPool2D(2,2))

model_3.add(layers.Flatten())

model_3.add(layers.Dense(128,
                       activation='relu'))

model_3.add(layers.Dense(3,
                       activation='softmax'))

In [ ]:
model_3.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history_3 = model_3.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    batch_size=128)

Epoch 1/10
56/56 [==============================] - 26s 334ms/step - loss: 1.1421 - accuracy: 0.4692 - val_loss: 1.0167 - val_accuracy: 0.4778
Epoch 2/10
56/56 [==============================] - 22s 361ms/step - loss: 0.9939 - accuracy: 0.5172 - val_loss: 0.9203 - val_accuracy: 0.6130
Epoch 3/10
56/56 [==============================] - 21s 337ms/step - loss: 0.8175 - accuracy: 0.6282 - val_loss: 0.8518 - val_accuracy: 0.6153
Epoch 4/10
56/56 [==============================] - 22s 364ms/step - loss: 0.7156 - accuracy: 0.6823 - val_loss: 0.7307 - val_accuracy: 0.6803
Epoch 5/10
56/56 [==============================] - 21s 338ms/step - loss: 0.6609 - accuracy: 0.7151 - val_loss: 0.7179 - val_accuracy: 0.6758
Epoch 6/10
56/56 [==============================] - 22s 345ms/step - loss: 0.5786 - accuracy: 0.7590 - val_loss: 0.7188 - val_accuracy: 0.6814
Epoch 7/10
56/56 [==============================] - 22s 337ms/step - loss: 0.4822 - accuracy: 0.8080 - val_loss: 0.7221 - val_accuracy: 0.6809

#### Second set

In [ ]:
# test_size = 0.2
# path = 'data'

# for sub_type in ['1', '2', '3']:
#   os.makedirs(path + '/test_augmented_2' +f'/{sub_type}', exist_ok=True)
#   file_list = os.listdir(path + '/train_augmented_2/' + sub_type)
#   number_of_samples = int(len(file_list) * test_size)

#   for file_name in random.sample(file_list, number_of_samples):
#     os.rename(path +'/train_augmented_2/' + sub_type + '/' + file_name,
#               path +'/test_augmented_2/' + sub_type + '/' + file_name)

# train_path = 'data/train_augmented_2'
# test_path = 'data/test_augmented_2'
# print(len(os.listdir(train_path + '/1')))
# print(len(os.listdir(test_path + '/1')))

6888
1722


##### Basic setup

In [ ]:
data_path = 'data/train_augmented_2'

batch_size = 128
img_heights = 128
img_width = 128



In [ ]:
train_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='training',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                    interpolation='bilinear',
                                                    seed=42)

val_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                  interpolation='bilinear',
                                                  seed=42)

Found 16194 files belonging to 3 classes.
Using 12956 files for training.
Found 16194 files belonging to 3 classes.
Using 3238 files for validation.


In [ ]:
model_4 = Sequential()

model_4.add(layers.Rescaling(1/255)) # przez to, z wczytujemy batchami trzeba ustandaryzować używajać warstwy Rescaling
model_4.add(layers.Conv2D(32,
                        (3, 3),
                        padding='same',
                        activation='relu',
                        input_shape=(img_width, img_heights, 3)))

model_4.add(layers.MaxPool2D(2,2))

model_4.add(layers.Conv2D(16,
                        (3, 3),
                        padding='same',
                        activation='relu'
                        ))

model_4.add(layers.MaxPool2D(2,2))

model_4.add(layers.Flatten())

model_4.add(layers.Dense(128,
                       activation='relu'))

model_4.add(layers.Dense(3,
                       activation='softmax'))

In [ ]:
model_4.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history_4 = model_4.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    batch_size=128)

Epoch 1/10
102/102 [==============================] - 45s 350ms/step - loss: 1.1136 - accuracy: 0.4265 - val_loss: 1.0497 - val_accuracy: 0.4219
Epoch 2/10
102/102 [==============================] - 39s 359ms/step - loss: 0.9821 - accuracy: 0.5185 - val_loss: 0.9862 - val_accuracy: 0.5022
Epoch 3/10
102/102 [==============================] - 41s 387ms/step - loss: 0.8484 - accuracy: 0.6078 - val_loss: 0.8283 - val_accuracy: 0.6072
Epoch 4/10
102/102 [==============================] - 39s 351ms/step - loss: 0.7281 - accuracy: 0.6767 - val_loss: 0.7647 - val_accuracy: 0.6510
Epoch 5/10
102/102 [==============================] - 39s 361ms/step - loss: 0.6311 - accuracy: 0.7343 - val_loss: 0.7735 - val_accuracy: 0.6566
Epoch 6/10
102/102 [==============================] - 39s 365ms/step - loss: 0.5381 - accuracy: 0.7810 - val_loss: 0.7493 - val_accuracy: 0.6773
Epoch 7/10
102/102 [==============================] - 39s 364ms/step - loss: 0.4555 - accuracy: 0.8199 - val_loss: 0.8042 - val_ac

##### More neurons

In [ ]:
data_path = 'data/train_augmented_2'

batch_size = 128
img_heights = 128
img_width = 128



In [ ]:
train_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='training',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                    interpolation='bilinear',
                                                    seed=42)

val_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                  interpolation='bilinear',
                                                  seed=42)

Found 16194 files belonging to 3 classes.
Using 12956 files for training.
Found 16194 files belonging to 3 classes.
Using 3238 files for validation.


In [ ]:
model_5 = Sequential()

model_5.add(layers.Rescaling(1/255)) # przez to, z wczytujemy batchami trzeba ustandaryzować używajać warstwy Rescaling
model_5.add(layers.Conv2D(64,
                        (3, 3),
                        padding='same',
                        activation='relu',
                        input_shape=(img_width, img_heights, 3)))

model_5.add(layers.MaxPool2D(2,2))

model_5.add(layers.Conv2D(32,
                        (3, 3),
                        padding='same',
                        activation='relu'
                        ))

model_5.add(layers.MaxPool2D(2,2))

model_5.add(layers.Flatten())

model_5.add(layers.Dense(256,
                       activation='relu'))

model_5.add(layers.Dense(3,
                       activation='softmax'))

In [ ]:
model_5.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history_5 = model_5.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    batch_size=128)

Epoch 1/10
102/102 [==============================] - 50s 387ms/step - loss: 1.2201 - accuracy: 0.4124 - val_loss: 1.0656 - val_accuracy: 0.4194
Epoch 2/10
102/102 [==============================] - 42s 397ms/step - loss: 1.0508 - accuracy: 0.4415 - val_loss: 1.0298 - val_accuracy: 0.4534
Epoch 3/10
102/102 [==============================] - 41s 378ms/step - loss: 0.8983 - accuracy: 0.5768 - val_loss: 0.8642 - val_accuracy: 0.5951
Epoch 4/10
102/102 [==============================] - 41s 379ms/step - loss: 0.7422 - accuracy: 0.6720 - val_loss: 0.8380 - val_accuracy: 0.6220
Epoch 5/10
102/102 [==============================] - 41s 379ms/step - loss: 0.6176 - accuracy: 0.7348 - val_loss: 0.8668 - val_accuracy: 0.6405
Epoch 6/10
102/102 [==============================] - 40s 376ms/step - loss: 0.4867 - accuracy: 0.7962 - val_loss: 0.8893 - val_accuracy: 0.6439
Epoch 7/10
102/102 [==============================] - 40s 376ms/step - loss: 0.3592 - accuracy: 0.8614 - val_loss: 0.9658 - val_ac

##### More layers

In [ ]:
data_path = 'data/train_augmented_2'

batch_size = 128
img_heights = 128
img_width = 128



In [ ]:
train_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='training',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                    interpolation='bilinear',
                                                    seed=42)

val_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                  interpolation='bilinear',
                                                  seed=42)

Found 16194 files belonging to 3 classes.
Using 12956 files for training.
Found 16194 files belonging to 3 classes.
Using 3238 files for validation.


In [ ]:
model_6 = Sequential()

model_6.add(layers.Rescaling(1/255)) # przez to, z wczytujemy batchami trzeba ustandaryzować używajać warstwy Rescaling
model_6.add(layers.Conv2D(32,
                        (3, 3),
                        padding='same',
                        activation='relu',
                        input_shape=(img_width, img_heights, 3)))

model_6.add(layers.MaxPool2D(2,2))

model_6.add(layers.Conv2D(16,
                        (3, 3),
                        padding='same',
                        activation='relu'
                        ))

model_6.add(layers.MaxPool2D(2,2))

model_6.add(layers.Conv2D(8,
                        (3, 3),
                        padding='same',
                        activation='relu'
                        ))

model_6.add(layers.MaxPool2D(2,2))

model_6.add(layers.Flatten())

model_6.add(layers.Dense(128,
                       activation='relu'))

model_6.add(layers.Dense(64,
                       activation='relu'))

model_6.add(layers.Dense(3,
                       activation='softmax'))

In [ ]:
model_6.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history_6 = model_6.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    batch_size=128)

Epoch 1/10
102/102 [==============================] - 46s 353ms/step - loss: 1.0674 - accuracy: 0.4270 - val_loss: 1.0554 - val_accuracy: 0.4203
Epoch 2/10
102/102 [==============================] - 40s 361ms/step - loss: 0.9530 - accuracy: 0.5408 - val_loss: 1.0615 - val_accuracy: 0.4901
Epoch 3/10
102/102 [==============================] - 39s 366ms/step - loss: 0.8076 - accuracy: 0.6268 - val_loss: 0.8127 - val_accuracy: 0.6177
Epoch 4/10
102/102 [==============================] - 39s 368ms/step - loss: 0.7360 - accuracy: 0.6652 - val_loss: 0.7767 - val_accuracy: 0.6448
Epoch 5/10
102/102 [==============================] - 39s 367ms/step - loss: 0.7026 - accuracy: 0.6837 - val_loss: 0.7277 - val_accuracy: 0.6754
Epoch 6/10
102/102 [==============================] - 40s 369ms/step - loss: 0.6347 - accuracy: 0.7205 - val_loss: 0.7017 - val_accuracy: 0.6936
Epoch 7/10
102/102 [==============================] - 42s 389ms/step - loss: 0.5889 - accuracy: 0.7476 - val_loss: 0.7446 - val_ac

##### Combination

In [ ]:
data_path = 'data/train_augmented_2'

batch_size = 128
img_heights = 128
img_width = 128



In [ ]:
train_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='training',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                    interpolation='bilinear',
                                                    seed=42)

val_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                  interpolation='bilinear',
                                                  seed=42)

Found 16194 files belonging to 3 classes.
Using 12956 files for training.
Found 16194 files belonging to 3 classes.
Using 3238 files for validation.


In [ ]:
model_7 = Sequential()

model_7.add(layers.Rescaling(1/255)) # przez to, z wczytujemy batchami trzeba ustandaryzować używajać warstwy Rescaling
model_7.add(layers.Conv2D(64,
                        (3, 3),
                        padding='same',
                        activation='relu',
                        input_shape=(img_width, img_heights, 3)))

model_7.add(layers.MaxPool2D(2,2))

model_7.add(layers.Conv2D(32,
                        (3, 3),
                        padding='same',
                        activation='relu'
                        ))

model_7.add(layers.MaxPool2D(2,2))

model_7.add(layers.Conv2D(16,
                        (3, 3),
                        padding='same',
                        activation='relu'
                        ))

model_7.add(layers.MaxPool2D(2,2))

model_7.add(layers.Flatten())

model_7.add(layers.Dense(256,
                       activation='relu'))

model_7.add(layers.Dense(128,
                       activation='relu'))

model_7.add(layers.Dense(3,
                       activation='softmax'))

In [ ]:
model_7.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history_7 = model_7.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    batch_size=128)

Epoch 1/10
102/102 [==============================] - 46s 360ms/step - loss: 1.0697 - accuracy: 0.4234 - val_loss: 1.0544 - val_accuracy: 0.4194
Epoch 2/10
102/102 [==============================] - 41s 365ms/step - loss: 0.9593 - accuracy: 0.5216 - val_loss: 1.0435 - val_accuracy: 0.4741
Epoch 3/10
102/102 [==============================] - 40s 365ms/step - loss: 0.7989 - accuracy: 0.6278 - val_loss: 0.9480 - val_accuracy: 0.5602
Epoch 4/10
102/102 [==============================] - 40s 368ms/step - loss: 0.7369 - accuracy: 0.6673 - val_loss: 0.7683 - val_accuracy: 0.6442
Epoch 5/10
102/102 [==============================] - 40s 372ms/step - loss: 0.6712 - accuracy: 0.7001 - val_loss: 0.7360 - val_accuracy: 0.6705
Epoch 6/10
102/102 [==============================] - 40s 372ms/step - loss: 0.5863 - accuracy: 0.7492 - val_loss: 0.7636 - val_accuracy: 0.6816
Epoch 7/10
102/102 [==============================] - 40s 373ms/step - loss: 0.5111 - accuracy: 0.7854 - val_loss: 0.7316 - val_ac

###### Tuning up with hyperparameters

In [8]:
data_path = 'data/train_augmented_2'

batch_size = 512
img_heights = 128
img_width = 128



In [9]:
train_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='training',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                    interpolation='bilinear',
                                                    seed=42)

val_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                  interpolation='bilinear',
                                                  seed=42)

Found 16194 files belonging to 3 classes.
Using 12956 files for training.
Found 16194 files belonging to 3 classes.
Using 3238 files for validation.


In [10]:
model_8 = Sequential()

model_8.add(layers.Rescaling(1/255)) # przez to, z wczytujemy batchami trzeba ustandaryzować używajać warstwy Rescaling
model_8.add(layers.Conv2D(64,
                        (3, 3),
                        padding='same',
                        activation='relu',
                        input_shape=(img_width, img_heights, 3)))

model_8.add(layers.MaxPool2D(2,2))

model_8.add(layers.Conv2D(32,
                        (3, 3),
                        padding='same',
                        activation='relu'
                        ))

model_8.add(layers.MaxPool2D(2,2))

model_8.add(layers.Conv2D(16,
                        (3, 3),
                        padding='same',
                        activation='relu'
                        ))

model_8.add(layers.MaxPool2D(2,2))

model_8.add(layers.Flatten())

model_8.add(layers.Dense(256,
                       activation='relu'))

model_8.add(layers.Dense(128,
                       activation='relu'))

model_8.add(layers.Dense(3,
                       activation='softmax'))

In [11]:
model_8.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

In [12]:
history_8 = model_8.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    batch_size=512)

Epoch 1/10
26/26 [==============================] - 53s 1s/step - loss: 1.0934 - accuracy: 0.4179 - val_loss: 1.0621 - val_accuracy: 0.4191
Epoch 2/10
26/26 [==============================] - 41s 1s/step - loss: 1.0632 - accuracy: 0.4308 - val_loss: 1.0569 - val_accuracy: 0.4614
Epoch 3/10
26/26 [==============================] - 41s 1s/step - loss: 1.0396 - accuracy: 0.4707 - val_loss: 1.0247 - val_accuracy: 0.4867
Epoch 4/10
26/26 [==============================] - 41s 1s/step - loss: 0.9953 - accuracy: 0.5198 - val_loss: 0.9373 - val_accuracy: 0.5639
Epoch 5/10
26/26 [==============================] - 41s 1s/step - loss: 0.9067 - accuracy: 0.5758 - val_loss: 0.8745 - val_accuracy: 0.6118
Epoch 6/10
26/26 [==============================] - 41s 1s/step - loss: 0.8277 - accuracy: 0.6206 - val_loss: 0.8625 - val_accuracy: 0.6072
Epoch 7/10
26/26 [==============================] - 41s 1s/step - loss: 0.7879 - accuracy: 0.6426 - val_loss: 0.7736 - val_accuracy: 0.6473
Epoch 8/10
26/26 [==